In [24]:
from pyspark.sql import SparkSession

import getpass

username = getpass.getuser()

spark = SparkSession.\
builder. \
config('spark.ui.port','0'). \
config("spark.sql.warehouse.dir", f"/user/{username}/warehouse"). \
enableHiveSupport(). \
master('yarn'). \
getOrCreate()

spark

In [25]:
orders_rdd = spark.sparkContext.textFile("/public/trendytech/orders/orders.csv")

In [20]:
rdd1 = orders_rdd.map(lambda x:(x.split(",")[-1],1))
rdd1.take(5)

[('CLOSED', 1),
 ('PENDING_PAYMENT', 1),
 ('COMPLETE', 1),
 ('CLOSED', 1),
 ('COMPLETE', 1)]

In [21]:
rdd2 = rdd1.reduceByKey(lambda x,y:x+y)


In [22]:
rdd2.collect()

[('PENDING', 9512500),
 ('PROCESSING', 10343750),
 ('CANCELED', 1785000),
 ('COMPLETE', 28623750),
 ('SUSPECTED_FRAUD', 1947500),
 ('CLOSED', 9445000),
 ('ON_HOLD', 4747500),
 ('PAYMENT_REVIEW', 911250),
 ('PENDING_PAYMENT', 18787500)]

In [31]:
mapped_rdd = orders_rdd.map(lambda x: (x.split(",")[-1],x.split(",")[-2]))

In [32]:
grouped_rdd = mapped_rdd.groupByKey()

In [33]:
fin_rdd = grouped_rdd.map(lambda x : (x[0],len(x[1])))

In [35]:
fin_rdd.collect()

[('PENDING', 9512500),
 ('PROCESSING', 10343750),
 ('CANCELED', 1785000),
 ('COMPLETE', 28623750),
 ('SUSPECTED_FRAUD', 1947500),
 ('CLOSED', 9445000),
 ('ON_HOLD', 4747500),
 ('PAYMENT_REVIEW', 911250),
 ('PENDING_PAYMENT', 18787500)]

In [36]:
spark.stop()